despite landing on a smooth surface, the box bounces / spins out of control after impact

In [ ]:
import brax
from google.protobuf import text_format
import jax

import numpy as np

from IPython.display import HTML, Image 
from brax.io import html
from brax import jumpy as jp

In [ ]:
# code from lunar lander env, specifics not relevant for this case
VIEWPORT_W = 600
VIEWPORT_H = 400

helipad_y = VIEWPORT_H / 4
SCALE = (VIEWPORT_H - helipad_y) / 24

CHUNKS = 11
W = VIEWPORT_W
H = VIEWPORT_H
height = np.random.uniform(0, H / 2, size=(CHUNKS + 1,))
chunk_x = [W / (CHUNKS - 1) * i for i in range(CHUNKS)]
helipad_x1 = chunk_x[CHUNKS // 2 - 1]
helipad_x2 = chunk_x[CHUNKS // 2 + 1]

height[CHUNKS // 2 - 2] = helipad_y
height[CHUNKS // 2 - 1] = helipad_y
height[CHUNKS // 2 + 0] = helipad_y
height[CHUNKS // 2 + 1] = helipad_y
height[CHUNKS // 2 + 2] = helipad_y
smooth_y = jp.array([
    0.33 * (height[i - 1] + height[i + 0] + height[i + 1])
    for i in range(CHUNKS)
])

brax_heightmap = jp.repeat(smooth_y / SCALE, CHUNKS).tolist()

In [ ]:
_CONFIG = """
  dt: 0.02 substeps: 4 friction: 0.98
  gravity { z: -5 }
  bodies {
    name: "lander" mass: 0.1
    colliders { box { halfsize { x: 1.7 y: 0.5 z: 1.4 }}}
    inertia { x: 1.0 y: 1.0 z: 1.0 }
  }
  bodies {
    name: "Ground"
    colliders {
      heightMap {
          size: %s
          data: %s
      }
    }
    frozen { all: true }
  }
  forces {
    name: "thruster"
    body: "lander"
    strength: 2.5
    thruster {}
  }
  frozen {
    position { y: 1.0 }
    rotation { x: 1.0 z: 1.0 }
  }
  defaults {
    qps { name: "lander" pos { z: 20 }}
    qps { name: "Ground" pos { x: -25 y: 25 z: 0 }}
  }
"""%(50, brax_heightmap,)

sys = brax.System(text_format.Parse(_CONFIG, brax.Config()))
step = jax.jit(sys.step)

In [ ]:
qp = sys.default_qp()
rollout = [qp]
for _ in range(200):
    qp, _ = step(qp, jp.zeros(3))
    rollout.append(qp)

In [ ]:
HTML(html.render(sys, rollout))